### 끝말잇기 게임

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

In [3]:
llm.invoke("대한민국의 수도는?")

AIMessage(content='대한민국의 수도는 **서울**입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--d5ca4798-5f1f-4c44-a83b-79102fd7d539-0', usage_metadata={'input_tokens': 7, 'output_tokens': 103, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 93}})

In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory


# 대화 기록을 저장할 히스토리 클래스 불러오기
chat_history = ChatMessageHistory()

chat_history.messages

[]

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 끝말잇기 게임을 진행하는 AI 챗봇입니다. 아래는 게임 규칙입니다. 당신과 user 의 입력에서 아래 규칙이 꼭 지켜져야 하며, 지키지 않은 사람에게 패배를 알린 뒤, 끝말잇기 게임을 종료합니다.
                1. 주어진 대화 기록에서 이미 나왔던 단어를 다시 말했을 경우 패배합니다.
                2. 두음법칙을 허용합니다. (ex. 리 -> 이, 력 -> 역, 락 -> 낙)
                3. 국어사전에 존재하는 단어이자, 명사여야 합니다.
                4. 아무런 설명 없이, 끝말잇기 단어만 한글로 한 단어만 출력하세요.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | llm

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [12]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        # return False
        return
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "위 채팅 메시지는 끝말잇기 게임을 진행한 대화내용입니다. 언급한 단어들만 나열하여 저장해주세요.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | llm

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    # return True
    return

chain_with_summarization = (
    # RunnablePassthrough는 LCEL에서 사용, 입력값을 다음 단계로 그대로 통과시키는 역할
    # assign() 메서드는 체인에 들어오는 딕셔너리에 새로운 키-값 추가
    RunnablePassthrough.assign(messages_summarized=summarize_messages) # 새로운 키 messages_summarized에 값(True 또는 False) 할당, 이 값은 조건부 요약에 활용 가능
    | chain_with_message_history
)

In [17]:
chain_with_summarization.invoke(
                {"input": "도시락"},
                {"configurable": {"session_id": "unused"}},
            )

AIMessage(content='낙엽', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--76a91b8a-9cc7-4ead-bf71-5d96319ac77d-0', usage_metadata={'input_tokens': 169, 'output_tokens': 156, 'total_tokens': 325, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 154}})

In [18]:
chat_history.messages

[AIMessage(content='기차\n차도', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--f4131b26-ecee-4266-8330-af25b550a2da-0', usage_metadata={'input_tokens': 37, 'output_tokens': 311, 'total_tokens': 348, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 306}}),
 HumanMessage(content='도시락', additional_kwargs={}, response_metadata={}),
 AIMessage(content='낙엽', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--76a91b8a-9cc7-4ead-bf71-5d96319ac77d-0', usage_metadata={'input_tokens': 169, 'output_tokens': 156, 'total_tokens': 325, 'input_token_details': {'cach

In [20]:
while True:
    user_input = input("🧑 YOUR TURN : ")
    if user_input == "종료": break
    response = chain_with_summarization.invoke(
                {"input": user_input},
                {"configurable": {"session_id": "unused"}},
            )
    print("🤖 AI TURN : ", response.content) # AIMessage 객체에서 .content 추출

🤖 AI TURN :  지구
🤖 AI TURN :  슬픔
🤖 AI TURN :  규칙 3을 위반하여 패배하셨습니다. 국어사전에 존재하는 단어가 아닙니다. 게임을 종료합니다.
